In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'SNP'

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_insync import util, IB
util.startLoop()

In [ ]:
# Set the root
from from_root import from_root
ROOT = from_root()

from utils import Vars
_vars = Vars(MARKET)
PORT = _vars.PORT
PAPER = _vars.PAPER 
OPT_COLS = _vars.OPT_COLS[0]
DATAPATH = ROOT / 'data' / MARKET

# RECTIFY PREPARE_SOW FUNCTION
## Customized `build_base` for a short number of symbols

In [ ]:
# Imports 
from utils import get_order_pf
import asyncio

In [ ]:
output = asyncio.run(get_order_pf(PAPER))

In [ ]:
df_order, df_pf = output

In [ ]:
df_order

In [ ]:
df_pf